In [ ]:
import pandas as pd
import seaborn as sns
import os

In [ ]:
dfs = []

bench_dir = "bench"

for cmd in os.listdir(bench_dir):
    for store_config in os.listdir(os.path.join(bench_dir, cmd)):
        print(f"processing {cmd}/{store_config}")
        parts = store_config.split(",")
        config = {}
        for part in parts:
            kv = part.split("=")
            config[kv[0]] = kv[1]

        store = config["name"]
        tls = config["tls"] == "True"
        enclave = "sgx" if config["sgx"] == "True" else "virtual"
        worker_threads = int(config["worker_threads"])
        
        file = os.path.join(bench_dir, cmd, store_config, "timings.csv")
        if not os.path.exists(file):
            continue
        df = pd.read_csv(file)
        start = df['start_micros'].min()
        df['start_micros'] -= start
        df['start_ms'] = df['start_micros'] / 1000
        df.drop(['start_micros'], axis=1, inplace=True)
        df['end_micros'] -= start
        df['end_ms'] = df['end_micros'] / 1000
        df.drop(['end_micros'], axis=1, inplace=True)
        df['latency_ms'] = (df['end_ms'] - df['start_ms'])
        df["cmd"] = cmd
        df["store"] = store
        df["tls"] = tls
        df["enclave"] = enclave
        df["worker_threads"] = worker_threads
        dfs.append(df)

all_data = pd.concat(dfs, ignore_index=True)
all_data.head()

In [ ]:
all_data[all_data.latency_ms < 0]

In [ ]:
all_data = all_data[all_data.latency_ms >= 0]
all_data = all_data[all_data.latency_ms < 500]

In [ ]:
all_data["store_tls"] = all_data["store"] + "_" + all_data["tls"].map(lambda t: "tls" if t else "plain")
all_data["store_tls_threads"] = all_data["store_tls"] + "_" + all_data["worker_threads"].map(lambda t: str(t))

In [ ]:
all_data["start_s"] = all_data["start_ms"] / 1000

In [ ]:
all_data

In [ ]:
# make the plots dir
os.makedirs("plots", exist_ok = True)

In [ ]:
p = sns.relplot(kind="scatter", data=all_data, x="start_s", y="latency_ms", row="cmd", col="enclave", hue="store_tls_threads", alpha=0.5)

# add tick labels to each x axis
for ax in p.axes.flatten():
    ax.tick_params(labelbottom=True)
    
#     ax.set_xlim([20,21])

p.savefig("plots/scatter.svg")

In [ ]:
p = sns.displot(kind="ecdf", data=all_data, x="latency_ms", row="cmd", col="enclave", hue="store_tls_threads")

# add tick labels to each x axis
for ax in p.axes.flatten():
    ax.tick_params(labelbottom=True)

p.savefig("plots/ecdf.svg")